# Pandas datetime object - Frequency Conversion (Resampling)

Frequency Conversion (Resampling) in Pandas refers to the process of changing the time frequency of your time series data. It's a fundamental operation in time series analysis, allowing you to adapt your data to a coarser or finer granularity.

Essentially, you're taking your existing data points and grouping them into new, larger or smaller time intervals, then deciding how to represent the data within those new intervals.

Here's a breakdown:

1. Core Purpose:

* To summarize data over longer periods (e.g., daily sales to monthly sales).
* To smooth out short-term fluctuations (e.g., high-frequency sensor readings to hourly averages).
* To align time series with different frequencies for comparison or joint analysis.
* To fill in missing time points.

2. Two Main Types of Resampling:

Downsampling (Higher Frequency to Lower Frequency):

* Concept: You are consolidating data points from a finer time interval into a coarser one (e.g., hourly to daily, daily to weekly, minute to hourly).
* Key Requirement: Because you are combining multiple data points into a single, new time point, downsampling always requires an aggregation function. You must tell Pandas how to summarize the data within each new, larger interval.
* Common Aggregation Functions:
  * .mean(): Calculate the average of all data points within the new interval.
  * .sum(): Sum all data points within the new interval.
  * .first(): Take the first data point within the new interval.
  * .last(): Take the last data point within the new interval.
  * .min(), .max(), .std(), .count(): Other statistical aggregations.
  * .ohlc(): For financial data, calculates Open, High, Low, Close.

Upsampling (Lower Frequency to Higher Frequency):

* Concept: You are expanding data points from a coarser time interval into a finer one (e.g., monthly to daily, daily to hourly).
* Key Challenge: When you create new, finer time points for which no original data existed, Pandas will initially fill these new entries with NaN (Not a Number).
* Handling Missing Values: You typically need to explicitly tell Pandas how to fill these NaN values:
  * .ffill() (Forward Fill): Propagates the last valid observation forward to fill the gaps. The value remains constant until the next original data point is encountered.
  * .bfill() (Backward Fill): Propagates the next valid observation backward to fill the gaps. The value is constant until the previous original data point is encountered.
  * .interpolate(): Estimates the values for the NaNs by looking at existing data points before and after. method='time' is often used for time series, performing linear interpolation based on the time intervals.

How it Works (Under the Hood, conceptually):

* The .resample() method (the primary function for frequency conversion) effectively performs a "group by" operation based on time intervals. It groups your time series data into bins defined by the new frequency rule (e.g., all data points in January, all data points in a specific week). Then, it applies the specified aggregation or filling logic to each bin.

Relationship with asfreq():

* While resample() is for both up and downsampling with aggregation, asfreq() is specifically for converting to a new frequency without aggregation. If asfreq() upsamples, it fills new time points with NaN by default, requiring subsequent fillna() or interpolate() calls.

# 1. Downsampling (Higher Frequency to Lower Frequency)
Downsampling involves aggregating data from a higher frequency (e.g., daily) to a lower frequency (e.g., weekly or monthly). This always requires an aggregation function (like mean(), sum(), first(), last(), ohlc(), etc.) to combine multiple data points into a single value for the new, coarser period.

### A. Create the dataset

In [66]:
import pandas as pd
import numpy as np

# Create a DataFrame with daily sales data for 30 days
daily_dates = pd.date_range(start='2023-01-01', periods=30, freq='D')
daily_sales = pd.DataFrame({
    'Sales': np.random.randint(50, 150, size=30),
    'Customers': np.random.randint(10, 30, size=30)
}, index=daily_dates)

In [67]:

print("Original Daily Sales Data (first 5 rows):\n")
daily_sales.head()


Original Daily Sales Data (first 5 rows):



,Sales,Customers
2023-01-01,83,12
2023-01-02,77,20
2023-01-03,141,26
2023-01-04,79,12
2023-01-05,66,17


In [68]:
print("Original Daily Sales Data (last 5 rows):\n")
daily_sales.tail()

Original Daily Sales Data (last 5 rows):



,Sales,Customers
2023-01-26,131,12
2023-01-27,92,18
2023-01-28,75,14
2023-01-29,78,10
2023-01-30,52,20


### B. Resample to Weekly Average Sales (week ends on Sunday by default)

In [69]:
# Resample to Weekly Average Sales (week ends on Sunday by default)

print("Weekly Average Sales (Sunday end)")
weekly_avg_sales = daily_sales.resample('W').mean()
weekly_avg_sales

Weekly Average Sales (Sunday end)


,Sales,Customers
2023-01-01,83.000000,12.000000
2023-01-08,98.285714,20.142857
2023-01-15,118.857143,21.285714
2023-01-22,111.285714,20.285714
2023-01-29,96.285714,17.285714
2023-02-05,52.000000,20.000000


### C. Resample to Weekly Sum of Sales (week ends on Friday)

In [70]:
# Resample to Weekly Sum of Sales (week ends on Friday)

print("Weekly Sum of Sales (Friday end)")
# 'W-FRI' specifies the week ending on Friday
weekly_sum_sales_fri = daily_sales.resample('W-FRI').sum()
weekly_sum_sales_fri

Weekly Sum of Sales (Friday end)


,Sales,Customers
2023-01-06,559,101
2023-01-13,796,157
2023-01-20,799,134
2023-01-27,749,149
2023-02-03,205,44


### D. Resample to Monthly Maximum Customers

In [73]:
# Resample to Monthly Maximum Customers

print("Monthly Maximum Customers")
# 'M' specifies month end frequency
monthly_max_customers = daily_sales.resample('ME')['Customers'].max()
monthly_max_customers

Monthly Maximum Customers


,Customers
2023-01-31,29


### E. Resample to Quarterly Open-High-Low-Close (OHLC) for Sales

In [74]:
# Resample to Quarterly Open-High-Low-Close (OHLC) for Sales

print("Quarterly OHLC Sales")
# 'Q' specifies quarter end frequency
quarterly_ohlc_sales = daily_sales['Sales'].resample('Q').ohlc()
quarterly_ohlc_sales

Quarterly OHLC Sales


<ipython-input-74-f41ef7c9d23d>:5: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  quarterly_ohlc_sales = daily_sales['Sales'].resample('Q').ohlc()


,open,high,low,close
2023-03-31,83,149,52,52


# 2. Upsampling (Lower Frequency to Higher Frequency)

Upsampling involves converting data from a lower frequency (e.g., monthly) to a higher frequency (e.g., daily or hourly). This process often introduces new rows with NaN values, as there's no inherent data for the newly created time points. You typically use fillna() methods like ffill() (forward-fill), bfill() (backward-fill), or interpolate() to manage these NaNs.

### A. Create the sample dataset

In [77]:
import pandas as pd
import numpy as np

print("Upsampling Monthly Data to Daily")

# Create a DataFrame with monthly revenue data for a few months
monthly_dates = pd.date_range(start='2023-01-31', periods=3, freq='ME')
monthly_revenue = pd.DataFrame({
    'Revenue': [5000, 5200, 5500]
}, index=monthly_dates)

Upsampling Monthly Data to Daily


In [79]:
monthly_revenue

,Revenue
2023-01-31,5000
2023-02-28,5200
2023-03-31,5500


### B. Implement forward fill

Forward Fill (ffill()) in upsampling is a method used to propagate the last valid observation forward to fill newly introduced missing values (NaNs) when you convert a time series from a lower frequency to a higher frequency.

Here's a precise explanation:

1. Upsampling Creates Gaps:

When you upsample a time series (e.g., converting monthly data to daily data), Pandas creates new rows or time points for which there was no original data. By default, the values for these new time points will be NaN (Not a Number).

Example: If you have data for January 31st and February 28th, and you upsample to daily, all the days from February 1st to February 27th will initially have NaN values.

2. How Forward Fill (ffill()) Works:

* ffill() systematically looks for the last non-NaN value in the series (or column) and then copies that value to subsequent NaN entries until it encounters another non-NaN value or reaches the end of the series. It literally "carries the last known value forward" in time.

Conceptual Example:

Original Monthly Data:

* Jan 31: 100
* Feb 28: 120

Upsampled to Daily (initial state with NaNs):

* Jan 31: 100
* Feb 01: NaN
* Feb 02: NaN
* ...
* Feb 27: NaN
* Feb 28: 120

After ffill():

* Jan 31: 100
* Feb 01: 100 (filled from Jan 31)
* Feb 02: 100 (filled from Jan 31)
* ...
* Feb 27: 100 (filled from Jan 31)
* Feb 28: 120 (original value, stops propagation of 100)

3. When to Use Forward Fill:

Forward fill is appropriate when:

* You expect the value to remain constant until the next recorded observation (e.g., daily stock prices, sensor readings that update periodically, or status indicators).
* You are dealing with cumulative data where the value doesn't decrease between recording points.
* You want to fill in gaps without introducing new calculated values (like interpolation).

In [94]:
print(" Daily Revenue (Forward Fill) ")

# 'D' specifies daily frequency
daily_revenue_ffill = monthly_revenue.resample('D').ffill()
print(daily_revenue_ffill.head()) # Shows filled values
print(daily_revenue_ffill.tail()) # Shows filled values

 Daily Revenue (Forward Fill) 
            Revenue
2023-01-31     5000
2023-02-01     5000
2023-02-02     5000
2023-02-03     5000
2023-02-04     5000
            Revenue
2023-03-27     5200
2023-03-28     5200
2023-03-29     5200
2023-03-30     5200
2023-03-31     5500


### C. Implement backward fill

Backward Fill (bfill()) in upsampling is a method used to propagate the next valid observation backward to fill newly introduced missing values (NaNs) when you convert a time series from a lower frequency to a higher frequency.

Here's a precise explanation:

1. Upsampling Creates Gaps:

As with forward fill, when you upsample a time series (e.g., converting weekly data to daily data), Pandas inserts new rows or time points for which there was no original data. The values for these new time points will be NaN.

Example: If you have data for January 1st and January 8th, and you upsample to daily, all the days from January 2nd to January 7th will initially have NaN values.

2. How Backward Fill (bfill()) Works:

bfill() systematically looks for the next non-NaN value in the series (or column) and then copies that value to preceding NaN entries until it encounters another non-NaN value or reaches the beginning of the series. It effectively "carries the next known value backward" in time.

Conceptual Example:

Original Weekly Data:

* Jan 01: 100
* Jan 08: 120

Upsampled to Daily (initial state with NaNs):

* Jan 01: 100
* Jan 02: NaN
* Jan 03: NaN
* ...
* Jan 07: NaN
* Jan 08: 120

After bfill():

* Jan 01: 100 (original value, stops propagation of 120)
* Jan 02: 120 (filled from Jan 08)
* Jan 03: 120 (filled from Jan 08)
* ...
* Jan 07: 120 (filled from Jan 08)
* Jan 08: 120

3. When to Use Backward Fill:

Backward fill is appropriate when:

* The data point represents a condition that will be true until the next recording (e.g., an end-of-period inventory count that applies backward through the period).
* You expect future observations to be more representative or available (e.g., in some forecasting or imputation scenarios where future data is known).
* You need to fill gaps in a way that respects a "future state" dependency.

In [93]:
# Upsample to Daily, filling NaNs with Backward Fill (bfill)

print(" Daily Revenue (Backward Fill) ")
daily_revenue_bfill = monthly_revenue.resample('D').bfill()
print(daily_revenue_bfill.head())
print(daily_revenue_bfill.tail())

 Daily Revenue (Backward Fill) 
            Revenue
2023-01-31     5000
2023-02-01     5200
2023-02-02     5200
2023-02-03     5200
2023-02-04     5200
            Revenue
2023-03-27     5500
2023-03-28     5500
2023-03-29     5500
2023-03-30     5500
2023-03-31     5500


### D. asfreq() directly converts the frequency


In [95]:
# Using .asfreq() for frequency conversion without aggregation
print(" Using .asfreq() for Daily Conversion ")
# asfreq() directly converts the frequency. Missing values will be NaN by default.
daily_revenue_asfreq = monthly_revenue.asfreq('D')
print(daily_revenue_asfreq.head()) # Shows NaNs where data was sparse
print(daily_revenue_asfreq.tail())

 Using .asfreq() for Daily Conversion 
            Revenue
2023-01-31   5000.0
2023-02-01      NaN
2023-02-02      NaN
2023-02-03      NaN
2023-02-04      NaN
            Revenue
2023-03-27      NaN
2023-03-28      NaN
2023-03-29      NaN
2023-03-30      NaN
2023-03-31   5500.0


### E. Using .asfreq() with interpolate()

When you use asfreq() with interpolate() for time series data, you are essentially performing a two-step process to handle missing values introduced by upsampling:

asfreq() (Creating the Gaps):

* The asfreq() method's primary role is to resample your time series to a new, specified frequency, without any aggregation. If you're upsampling (going from a lower frequency like monthly to a higher frequency like daily), asfreq() will create new entries in your time series for every time point in the new, higher frequency. For all these newly added time points where no original data existed, asfreq() will fill the values with NaN. It lays out the complete, new-frequency timeline, exposing all the missing observations.

interpolate() (Filling the Gaps with Estimation):

* Once asfreq() has established the desired frequency and populated any newly created time points with NaN, the interpolate() method comes into play. Unlike ffill() or bfill() which simply copy existing values, interpolate() estimates the values for these NaNs.
    * For time series data, using interpolate(method='time') is particularly powerful. This method performs linear interpolation based on the time index. It looks at the valid data points before and after a NaN and then calculates a value for the NaN based on its position in time between those two known points. This creates a smoother transition of values, as opposed to the step-like changes produced by forward or backward fill.

General Concept of asfreq() with interpolate():

* You use asfreq() first to ensure your time series has the exact frequency you desire. This step will inherently introduce NaNs for all the time points that didn't exist in your original, coarser dataset. Then, you chain interpolate(method='time') to these NaN-filled gaps. This second step doesn't just copy the last or next known value; instead, it mathematically estimates a plausible value for each missing point, assuming a relatively continuous change between the existing data points. This approach is beneficial when you want to infer intermediate values that reflect a trend or a smooth progression over time, rather than a discrete state change.

In [97]:
# You can combine asfreq with fillna

print(" Using .asfreq() with interpolate() ---")

# Interpolate fills NaNs by estimating values between known points
daily_revenue_interp = monthly_revenue.asfreq('D').interpolate(method='time')
print(daily_revenue_interp.head(7)) # Show more rows to see interpolation effect

 Using .asfreq() with interpolate() ---
                Revenue
2023-01-31  5000.000000
2023-02-01  5007.142857
2023-02-02  5014.285714
2023-02-03  5021.428571
2023-02-04  5028.571429
2023-02-05  5035.714286
2023-02-06  5042.857143
